<a href="https://colab.research.google.com/github/Jungin1020/Aiffel_exp/blob/main/2022_03_01_exp_14_recommendation_Movielens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##데이터 불러오기

In [67]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [68]:
! wget -p /content/drive/MyDrive/aiffel/recommendata_iu/data http://files.grouplens.org/datasets/movielens/ml-1m.zip
! cd /content/drive/MyDrive/aiffel/recommendata_iu/data


/content/drive/MyDrive/aiffel/recommendata_iu/data: Scheme missing.
--2022-03-01 12:50:38--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘files.grouplens.org/datasets/movielens/ml-1m.zip’

files.grouplens.org 100%[===================>]   5.64M  4.14MB/s    in 1.4s    

2022-03-01 12:50:40 (4.14 MB/s) - ‘files.grouplens.org/datasets/movielens/ml-1m.zip’ saved [5917549/5917549]

FINISHED --2022-03-01 12:50:40--
Total wall clock time: 1.9s
Downloaded: 1 files, 5.6M in 1.4s (4.14 MB/s)


In [69]:
! unzip /content/drive/MyDrive/aiffel/recommendata_iu/data/ml-1m.zip

Archive:  /content/drive/MyDrive/aiffel/recommendata_iu/data/ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


##데이터 전처리

In [70]:
import pandas as pd
import os

rating_file_path='/content/drive/MyDrive/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names= ratings_cols, engine='python', encoding='ISO-8859-1')
original_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


###3점 이상만 남기기

In [71]:
ratings = ratings[ratings['ratings'] >= 3] 
filtered_data_size = len(ratings)

print(f'original_data_size: {original_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio or Remaining Data is {filtered_data_size/original_data_size:.2%}')

original_data_size: 1000209, filtered_data_size: 836478
Ratio or Remaining Data is 83.63%


###컬럼 이름 바꾸기

In [72]:
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [73]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [74]:
movie_file_path = '/content/drive/MyDrive/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [75]:
ratings = pd.merge(ratings, movies[['title', 'movie_id']], on='movie_id', how='left')
ratings

,user_id,movie_id,counts,timestamp,title
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,978302109,James and the Giant Peach (1996)
2,1,914,3,978301968,My Fair Lady (1964)
3,1,3408,4,978300275,Erin Brockovich (2000)
4,1,2355,5,978824291,"Bug's Life, A (1998)"
...,...,...,...,...,...
836473,6040,1090,3,956715518,Platoon (1986)
836474,6040,1094,5,956704887,"Crying Game, The (1992)"
836475,6040,562,5,956704746,Welcome to the Dollhouse (1995)
836476,6040,1096,4,956715648,Sophie's Choice (1982)


In [76]:
movies['title'] = movies['title'].str.lower()
movies

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy
...,...,...,...
3878,3948,meet the parents (2000),Comedy
3879,3949,requiem for a dream (2000),Drama
3880,3950,tigerland (2000),Drama
3881,3951,two family house (2000),Drama


In [77]:
movies['title'][3840:3870]

3840                            dancer in the dark (2000)
3841                                  best in show (2000)
3842                                     beautiful (2000)
3843                          barenaked in america (1999)
3844                       broken hearts club, the (2000)
3845                                     girlfight (2000)
3846                           remember the titans (2000)
3847                                    hellraiser (1987)
3848                      hellbound: hellraiser ii (1988)
3849                 hellraiser iii: hell on earth (1992)
3850    faraway, so close (in weiter ferne, so nah!) (...
3851                                   beach party (1963)
3852                                  bikini beach (1964)
3853                             return of the fly (1959)
3854                                  pajama party (1964)
3855                        stranger than paradise (1984)
3856               voyage to the bottom of the sea (1961)
3857          

##데이터 분석

In [78]:
movies['movie_id'].nunique()

3883

In [79]:
ratings['movie_id'].nunique()

3628

In [80]:
ratings['user_id'].nunique()

6039

가장 인기 많은 영화 10개

In [81]:
movie_count = ratings.groupby('movie_id')['counts'].count()
movie_count.sort_values(ascending=False).head(10)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
Name: counts, dtype: int64

In [82]:
movies.loc[2789]

movie_id                      2858
title       american beauty (1999)
genre                 Comedy|Drama
Name: 2789, dtype: object

타이틀에 연도까지 적혀있어서 내가 선호하는 리스트 추가가 쉽지 않겠다...


.str.contains()로 내가 원하는 영화 찾기

In [83]:
movies[movies['title'].str.contains('mission')]

,movie_id,title,genre
642,648,mission: impossible (1996),Action|Adventure|Mystery
2676,2745,"mission, the (1986)",Drama
3285,3354,mission to mars (2000),Sci-Fi
3554,3623,mission: impossible 2 (2000),Action|Thriller
3817,3887,went to coney island on a mission from god... ...,Drama


In [84]:
movies[movies['title'].str.contains('sleeping')]

,movie_id,title,genre
335,339,while you were sleeping (1995),Comedy|Romance
2027,2096,sleeping beauty (1959),Animation|Children's|Musical


In [85]:
movies[movies['title'].str.contains('titanic')]

,movie_id,title,genre
1672,1721,titanic (1997),Drama|Romance
2088,2157,"chambermaid on the titanic, the (1998)",Romance
3334,3403,raise the titanic (1980),Drama|Thriller
3335,3404,titanic (1953),Action|Drama


In [86]:
movies[movies['title'].str.contains('ripley')]

,movie_id,title,genre
3107,3176,"talented mr. ripley, the (1999)",Drama|Mystery|Thriller


##내가 선호하는 영화 5개 rating에 추가

In [87]:
my_favorite = ['titanic (1997)', 'sleeping beauty (1959)', 'mission: impossible (1996)', 'Toy Story (1995)', 'talented mr. ripley, the (1999)']
my_list = pd.DataFrame({'user_id' : ['jung']*5, 'title': my_favorite, 'counts' : [5]*5})
if not ratings.isin({'user_id':['jung']})['user_id'].any():
  ratings = ratings.append(my_list)

ratings.tail(10)

,user_id,movie_id,counts,timestamp,title
836473,6040,1090.0,3,956715518.0,Platoon (1986)
836474,6040,1094.0,5,956704887.0,"Crying Game, The (1992)"
836475,6040,562.0,5,956704746.0,Welcome to the Dollhouse (1995)
836476,6040,1096.0,4,956715648.0,Sophie's Choice (1982)
836477,6040,1097.0,4,956715569.0,E.T. the Extra-Terrestrial (1982)
0,jung,NaN,5,NaN,titanic (1997)
1,jung,NaN,5,NaN,sleeping beauty (1959)
2,jung,NaN,5,NaN,mission: impossible (1996)
3,jung,NaN,5,NaN,Toy Story (1995)
4,jung,NaN,5,NaN,"talented mr. ripley, the (1999)"


In [88]:
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [91]:
print(user_to_idx['jung'])
print(movie_to_idx['titanic (1997)'])

6039
3628


In [92]:
print(len(ratings['user_id'].unique()))

6040


In [93]:
#user_to_idx에서 얻은 index를 user_id에 매핑, 결측치 없애기
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()

if len(temp_user_data) == len(ratings):
  print('user id column indexing OK')
  ratings['user_id'] = temp_user_data

else:
  print('user id column indexing FAIL')

temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
  print('movie_id column indexing OK')
  ratings['movie_id'] = temp_movie_data

else:
  print('movie_id column indexing FAIL')

ratings

user id column indexing OK
movie_id column indexing OK


,user_id,movie_id,counts,timestamp,title
0,0,0,5,978300760.0,One Flew Over the Cuckoo's Nest (1975)
1,0,1,3,978302109.0,James and the Giant Peach (1996)
2,0,2,3,978301968.0,My Fair Lady (1964)
3,0,3,4,978300275.0,Erin Brockovich (2000)
4,0,4,5,978824291.0,"Bug's Life, A (1998)"
...,...,...,...,...,...
0,6039,3628,5,NaN,titanic (1997)
1,6039,3629,5,NaN,sleeping beauty (1959)
2,6039,3630,5,NaN,mission: impossible (1996)
3,6039,40,5,NaN,Toy Story (1995)


##CSR matrix

In [94]:
#csr data - sparse한 데이터 변환

from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['counts'], (ratings.user_id, ratings.movie_id)), shape=(num_user, num_movie))
csr_data

<6040x3632 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

class 'numpy.longlong' 이게 뭘까?

예제에서는 int64가 뜬거 같은데..

In [95]:
!pip install implicit
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

     |████████████████████████████████| 18.5 MB 437 kB/s 


/usr/local/lib/python3.7/dist-packages/implicit/gpu/__init__.py:14: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  f"CUDA extension is built, but disabling GPU support because of '{e}'",


##als 모델 구현

In [99]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=50, dtype=np.float32)

1. factors: 유저와 아이템의 벡터를 몇 차원으로 할 건지
2. regularization: 정규화값을 얼마나 사용할 것인지
3. iterations: epochs와 같은 의미

In [100]:
csr_data_transpose = csr_data.T
csr_data_transpose

<3632x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

als 모델은 item x user 꼴의 metrix를  input으로 받기 때문에 Transpose 해준다

In [101]:
als_model.fit(csr_data_transpose)

  0%|          | 0/50 [00:00<?, ?it/s]

##영화 선호도 예측하기

내가 좋아하는 영화와 선호도 예측하기

In [103]:
jung, titanic = user_to_idx['jung'], movie_to_idx['titanic (1997)']
jung_vector, titanic_vector = als_model.user_factors[jung], als_model.item_factors[titanic]

IndexError: ignored

연습할 떄도 났던 오류

.user_factors 와 .item_factors 가 반대로 들어가는 것 같다?

In [104]:
jung, titanic = user_to_idx['jung'], movie_to_idx['titanic (1997)']
jung_vector, titanic_vector = als_model.item_factors[jung], als_model.user_factors[titanic]

In [105]:
np.dot(jung_vector, titanic_vector)

0.0023173024

??? 왜이리 작게 나올까

In [106]:
toy = movie_to_idx['Toy Story (1995)']
toy_vector = als_model.user_factors[toy]
np.dot(jung_vector, toy_vector)

0.44647658

이건 그래도 어느정도 높게 나왔다

##내가 좋아하는 영화와 비슷한 영화 추천 받기

In [111]:
favorite_movie = 'Toy Story (1995)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15) #N은 개수
similar_movie

(array([  40, 2762,   85, 5278, 3406, 5358, 5695, 5638,  865,  420,   63,
        1595, 5670,  310, 5861], dtype=int32),
 array([0.9999999 , 0.66450846, 0.5466851 , 0.53568953, 0.5298144 ,
        0.52591914, 0.51297474, 0.5034221 , 0.50225437, 0.49714577,
        0.490527  , 0.4893644 , 0.4866142 , 0.48130384, 0.4802076 ],
       dtype=float32))

array가 이상하게 나왔다

In [114]:
a,b = similar_movie[0],similar_movie[1]
a

array([  40, 2762,   85, 5278, 3406, 5358, 5695, 5638,  865,  420,   63,
       1595, 5670,  310, 5861], dtype=int32)

이게 추천 받은 영화 리스트인데 id를 title로 변환할 수 있을까?

In [118]:
#movie_to_idx 를 뒤집어 index로부터 movie 이름을 얻는 dict 생성
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
#movie id를 다시 movie 이름으로 맵핑
[idx_to_movie[i] for i in similar_movie]

TypeError: ignored

In [121]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie[0]]

IndexError: ignored

In [141]:
ratings[ratings['movie_id'] == a[0]]['title'].unique()

array(['Toy Story (1995)'], dtype=object)

In [143]:
ratings[ratings['movie_id'] == a[3]]['title']

Series([], Name: title, dtype: object)

In [142]:
for i in range(15):
  df = ratings[ratings['movie_id'] == a[i]]['title'].unique()
  print(df)

['Toy Story (1995)']
['Voyage of the Damned (1976)']
['League of Their Own, A (1992)']
[]
["Wooden Man's Bride, The (Wu Kui) (1994)"]
[]
[]
[]
['Predator (1987)']
['Heaven & Earth (1993)']
['Hustler, The (1961)']
['Amityville Horror, The (1979)']
[]
['City of Lost Children, The (1995)']
[]


array 형태 때문에 계속 오류가 나서 다른 방식으로 뽑았다.

안나오는 애들은 뭘까?

##회고


- 내가 좋아하는 영화로 못 박은 영화도 선호도가 낮게 나와서 희한했다
- 시작부터 가까운 벡터로 설정되어야하는 것 아닌가?
- .user_factors 와 .item_factors 가 반대로 들어갔을 때 잘 작동되었던게 제일 찝찝하다
- 다음주 데이터톤에서 데이터프레임 엄청 다뤄야하는데 미숙해서 걱정이다ㅜ